## GluonCV on SageMaker demo

Let's first import the necessary modules. 

In [1]:
%matplotlib inline
import numpy as np
import cv2
import boto3
import json
import mxnet as mx
from gluoncv import model_zoo, data, utils
import matplotlib.pyplot as plt

### Object Detection with a pre-trained SSD model

In this demo we are going to perform inference on a GluonCV object detection model that has been deployed in Amazon SageMaker. We run this notebook on our laptop to capture image data from the webcam and to send it to the SageMaker endpoint. 

The endpoint is running the model `ssd_512_resnet50_v1_voc` which has the following structure:

In [ ]:
net = model_zoo.get_model('ssd_512_resnet50_v1_voc', pretrained=False)
net

At this step we assume that the AWS credentials have been set beforehand. If this is not the case, make sure to set the following environment variables in the commandline before starting this notebook: `AWS_ACCESS_KEY_ID` `AWS_SECRET_ACCESS_KEY` and `AWS_SESSION_TOKEN`.

In the following lines of code, we list the endpoints that have been deployed in our account.

In [ ]:
runtime= boto3.client('runtime.sagemaker')
client = boto3.client('sagemaker')
client.list_endpoints()['Endpoints']

### Pretrained SSD model

Next we get the endpoint that has been deployed under the name `gluoncv-pretrained-object-detection-ssd`

In [4]:
for endpoint in  client.list_endpoints()['Endpoints']:
    if endpoint["EndpointName"] == "gluoncv-pretrained-object-detection-ssd":
        gluoncv_endpoint = endpoint

Now we capture image data from the local webcam and send it to the endpoint. The endpoint will perform inference and return bounding boxes, class ids and confidence scores.

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    img = cv2.resize(frame, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=net.classes)
    
    cv2.imshow("image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### AWS Logo detection

We have deployed the GluonCV object detection model before in SageMaker under the name `gluoncv-demo-object-detection`. Now we retrieve this endpoint.

In [6]:
for endpoint in  client.list_endpoints()['Endpoints']:
    if endpoint["EndpointName"] == "gluoncv-aws-logo-detection":
        gluoncv_endpoint = endpoint

In the following code section, we retrieve frames from the webcam and send it to the SageMaker endpoint. We first have to convert the data into JSON format.

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    img = cv2.resize(frame, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=["AWS"])
    
    cv2.imshow("image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()